# VTL Simple analytic function



In [1]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType, LongType, DecimalType
import os
from pyspark.sql.functions import lit, count,sum,avg,collect_list,min,max,percentile_approx,stddev_pop,stddev_samp,var_pop,var_samp
from pyspark.sql.window import Window

In [2]:
local = True

if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("VTLAnalytic")\
        .getOrCreate()
else:
    spark = SparkSession.builder\
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("VTLAnalytic")\
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:py3.9.7-spark3.2.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "8g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-07-11 12:50:42,630 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data=[("A", "XX", 2000, 3, 1.0),
    ("A", "XX", 2001, 4, 9.0),
    ("A", "XX", 2002, 7, 5.0),
    ("A", "XX", 2003, 6, 8.0),
    ("A", "YY", 2000, 9, 3.0),
    ("A", "YY", 2001, 5, 4.0),
    ("A", "YY", 2002, 10, 2.0),
    ("A", "YY", 2003, 5, 7.0)]

schema=StructType([StructField("Id_1",StringType(),True),
                   StructField("Id_2",StringType(),True),
                   StructField("Year",IntegerType(),True),
                   StructField("Me_1",IntegerType(),True),
                   StructField("Me_2",DoubleType(),True)])

df=spark.createDataFrame(data, schema)
df.show()

+----+----+----+----+----+
|Id_1|Id_2|Year|Me_1|Me_2|
+----+----+----+----+----+
|   A|  XX|2000|   3| 1.0|
|   A|  XX|2001|   4| 9.0|
|   A|  XX|2002|   7| 5.0|
|   A|  XX|2003|   6| 8.0|
|   A|  YY|2000|   9| 3.0|
|   A|  YY|2001|   5| 4.0|
|   A|  YY|2002|  10| 2.0|
|   A|  YY|2003|   5| 7.0|
+----+----+----+----+----+



## 1.1 Count

### 1.1.1 Exp1 : window only has partition clause

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 ) )
```


In [11]:
partition_col_name="Id_1"

 
win_name=Window.partitionBy(partition_col_name)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"count_{target_col_name1}"
new_col_name2=f"count_{target_col_name2}"
df_count=df.withColumn(new_col_name1,count(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name))
df_count.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         8|         8|
|   A|  XX|2001|   4| 9.0|         8|         8|
|   A|  XX|2002|   7| 5.0|         8|         8|
|   A|  XX|2003|   6| 8.0|         8|         8|
|   A|  YY|2000|   9| 3.0|         8|         8|
|   A|  YY|2001|   5| 4.0|         8|         8|
|   A|  YY|2002|  10| 2.0|         8|         8|
|   A|  YY|2003|   5| 7.0|         8|         8|
+----+----+----+----+----+----------+----------+



### 1.1.2 Exp 2: window has partition and order by

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Id_2) )
```


In [12]:

win_name_with_order=Window.partitionBy(partition_col_name).orderBy("Id_2")

df_count_with_order=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order))

df_count_with_order.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         4|         4|
|   A|  XX|2001|   4| 9.0|         4|         4|
|   A|  XX|2002|   7| 5.0|         4|         4|
|   A|  XX|2003|   6| 8.0|         4|         4|
|   A|  YY|2000|   9| 3.0|         8|         8|
|   A|  YY|2001|   5| 4.0|         8|         8|
|   A|  YY|2002|  10| 2.0|         8|         8|
|   A|  YY|2003|   5| 7.0|         8|         8|
+----+----+----+----+----+----------+----------+



### 1.1.3 Exp 3: window has partition, order by and data point

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [14]:
win_name_with_order_dp=Window.partitionBy(partition_col_name).orderBy("Id_2").rowsBetween(-2,2)

df_count_with_order_dp=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order_dp))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order_dp))

df_count_with_order_dp.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         3|         3|
|   A|  XX|2001|   4| 9.0|         4|         4|
|   A|  XX|2002|   7| 5.0|         5|         5|
|   A|  XX|2003|   6| 8.0|         5|         5|
|   A|  YY|2000|   9| 3.0|         5|         5|
|   A|  YY|2001|   5| 4.0|         5|         5|
|   A|  YY|2002|  10| 2.0|         4|         4|
|   A|  YY|2003|   5| 7.0|         3|         3|
+----+----+----+----+----+----------+----------+



### 1.1.4 Exp 4: window has partition, order by and range 

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [5]:
win_name_with_order_range=Window.partitionBy(partition_col_name).orderBy("Year").rangeBetween(-2,2)

df_count_with_order_range=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order_range))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order_range))

df_count_with_order_range.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         6|         6|
|   A|  YY|2000|   9| 3.0|         6|         6|
|   A|  XX|2001|   4| 9.0|         8|         8|
|   A|  YY|2001|   5| 4.0|         8|         8|
|   A|  XX|2002|   7| 5.0|         8|         8|
|   A|  YY|2002|  10| 2.0|         8|         8|
|   A|  XX|2003|   6| 8.0|         6|         6|
|   A|  YY|2003|   5| 7.0|         6|         6|
+----+----+----+----+----+----------+----------+



In [6]:
df.printSchema()

root
 |-- Id_1: string (nullable = true)
 |-- Id_2: string (nullable = true)
 |-- Id_3: integer (nullable = true)
 |-- Me_1: integer (nullable = true)
 |-- Me_2: integer (nullable = true)



### 1.1.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc count_Me_1:= count ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [14]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
 
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)
target_col_name1="Me_1"
new_col_name1=f"count_{target_col_name1}"
df_count=df.withColumn(new_col_name1,count(target_col_name1).over(win_name))
df_count.show()



+----+----+----+----+----+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|
+----+----+----+----+----+----------+
|   A|  XX|2000|   3| 1.0|         1|
|   A|  XX|2001|   4| 9.0|         2|
|   A|  XX|2002|   7| 5.0|         3|
|   A|  XX|2003|   6| 8.0|         4|
|   A|  YY|2000|   9| 3.0|         1|
|   A|  YY|2001|   5| 4.0|         2|
|   A|  YY|2002|  10| 2.0|         3|
|   A|  YY|2003|   5| 7.0|         4|
+----+----+----+----+----+----------+



## 1.2 Sum

### 1.2.1 Exp1 : window only has partition clause
The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1, Id_2 ) )
```


In [10]:
partition_col_name=["Id_1","Id_2"]
win_name=Window.partitionBy(partition_col_name)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      20|    23.0|
|   A|  XX|2001|   4| 9.0|      20|    23.0|
|   A|  XX|2002|   7| 5.0|      20|    23.0|
|   A|  XX|2003|   6| 8.0|      20|    23.0|
|   A|  YY|2000|   9| 3.0|      29|    16.0|
|   A|  YY|2001|   5| 4.0|      29|    16.0|
|   A|  YY|2002|  10| 2.0|      29|    16.0|
|   A|  YY|2003|   5| 7.0|      29|    16.0|
+----+----+----+----+----+--------+--------+



### 1.2.2 Exp 2: window has partition and order by

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Id_2) )
```

In [32]:
partition_col_name="Id_1"
win_name=Window.partitionBy(partition_col_name).orderBy("Id_2")

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      20|    23.0|
|   A|  XX|2001|   4| 9.0|      20|    23.0|
|   A|  XX|2002|   7| 5.0|      20|    23.0|
|   A|  XX|2003|   6| 8.0|      20|    23.0|
|   A|  YY|2000|   9| 3.0|      49|    39.0|
|   A|  YY|2001|   5| 4.0|      49|    39.0|
|   A|  YY|2002|  10| 2.0|      49|    39.0|
|   A|  YY|2003|   5| 7.0|      49|    39.0|
+----+----+----+----+----+--------+--------+



### 1.2.3 Window with orderby without partition, 

The vtl query:

```text
res := sum ( ds1 over ( order by Id_1, Id_2, Year ) )
``` 

As there is no partition, so the whole data frame is considered as one partition. So the sum become the rolling sum of the whole data set.
This is not recommended, because in spark, one partition means only one worker will do the job, the rest of the worker in the cluster will be idle. It's no longer distributed calculation.

In [33]:
partition_col_names=["Id_1","Id_2","Year"]
win_name=Window.orderBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       7|    10.0|
|   A|  XX|2002|   7| 5.0|      14|    15.0|
|   A|  XX|2003|   6| 8.0|      20|    23.0|
|   A|  YY|2000|   9| 3.0|      29|    26.0|
|   A|  YY|2001|   5| 4.0|      34|    30.0|
|   A|  YY|2002|  10| 2.0|      44|    32.0|
|   A|  YY|2003|   5| 7.0|      49|    39.0|
+----+----+----+----+----+--------+--------+



2022-06-29 13:19:44,373 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-29 13:19:44,373 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-29 13:19:44,472 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### 1.2.4 Exp : window has partition, order by and data point

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [34]:
partition_col_name=["Id_1"]
order_col_name=["Id_2"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_col_name).rowsBetween(-2,2)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      14|    15.0|
|   A|  XX|2001|   4| 9.0|      20|    23.0|
|   A|  XX|2002|   7| 5.0|      29|    26.0|
|   A|  XX|2003|   6| 8.0|      31|    29.0|
|   A|  YY|2000|   9| 3.0|      37|    22.0|
|   A|  YY|2001|   5| 4.0|      35|    24.0|
|   A|  YY|2002|  10| 2.0|      29|    16.0|
|   A|  YY|2003|   5| 7.0|      20|    13.0|
+----+----+----+----+----+--------+--------+



### 1.2.5 Exp : window has partition, order by and range 

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [35]:
partition_col_name=["Id_1"]
order_col_name=["Year"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_col_name).rangeBetween(-1,1)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
                     
df_sum.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      21|    17.0|
|   A|  YY|2000|   9| 3.0|      21|    17.0|
|   A|  XX|2001|   4| 9.0|      38|    24.0|
|   A|  YY|2001|   5| 4.0|      38|    24.0|
|   A|  XX|2002|   7| 5.0|      37|    35.0|
|   A|  YY|2002|  10| 2.0|      37|    35.0|
|   A|  XX|2003|   6| 8.0|      28|    22.0|
|   A|  YY|2003|   5| 7.0|      28|    22.0|
+----+----+----+----+----+--------+--------+



### 1.2.6 Exp6: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc sum_Me_1:= sum ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [15]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
 
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)
target_col_name1="Me_1"
new_col_name1=f"sum_{target_col_name1}"
df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name))
df_sum.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|
+----+----+----+----+----+--------+
|   A|  XX|2000|   3| 1.0|       3|
|   A|  XX|2001|   4| 9.0|       7|
|   A|  XX|2002|   7| 5.0|      14|
|   A|  XX|2003|   6| 8.0|      20|
|   A|  YY|2000|   9| 3.0|       9|
|   A|  YY|2001|   5| 4.0|      14|
|   A|  YY|2002|  10| 2.0|      24|
|   A|  YY|2003|   5| 7.0|      29|
+----+----+----+----+----+--------+



## 1.3 min

### 1.3.1 Exp1 : window only has partition clause

The vtl query:

```text
res := min ( ds1 over ( partition by Id_1, Id_2 ) )
```



In [94]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"min_{target_col_name1}"
new_col_name2=f"min_{target_col_name2}"

df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,min(target_col_name2).over(win_name))
                     
df_min.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|min_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       3|     1.0|
|   A|  XX|2002|   7| 5.0|       3|     1.0|
|   A|  XX|2003|   6| 8.0|       3|     1.0|
|   A|  YY|2000|   9| 3.0|       5|     2.0|
|   A|  YY|2001|   5| 4.0|       5|     2.0|
|   A|  YY|2002|  10| 2.0|       5|     2.0|
|   A|  YY|2003|   5| 7.0|       5|     2.0|
+----+----+----+----+----+--------+--------+



### 1.3.2 Exp 2: window has partition and order by

The vtl query:

```text
res := min ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [95]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy("Year")

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"min_{target_col_name1}"
new_col_name2=f"min_{target_col_name2}"

df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,min(target_col_name2).over(win_name))
                     
df_min.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|min_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       3|     1.0|
|   A|  XX|2002|   7| 5.0|       3|     1.0|
|   A|  XX|2003|   6| 8.0|       3|     1.0|
|   A|  YY|2000|   9| 3.0|       9|     3.0|
|   A|  YY|2001|   5| 4.0|       5|     3.0|
|   A|  YY|2002|  10| 2.0|       5|     2.0|
|   A|  YY|2003|   5| 7.0|       5|     2.0|
+----+----+----+----+----+--------+--------+



### 1.3.3 Window with orderby without partition, 

The vtl query:

```text
res := min ( ds1 over ( order by Id_1, Id_2, Year ) )
``` 


In [99]:
order_col_names=["Id_1","Id_2","Year"]

win_name=Window.orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"min_{target_col_name1}"
new_col_name2=f"min_{target_col_name2}"

df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,min(target_col_name2).over(win_name))
                     
df_min.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|min_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       3|     1.0|
|   A|  XX|2002|   7| 5.0|       3|     1.0|
|   A|  XX|2003|   6| 8.0|       3|     1.0|
|   A|  YY|2000|   9| 3.0|       3|     1.0|
|   A|  YY|2001|   5| 4.0|       3|     1.0|
|   A|  YY|2002|  10| 2.0|       3|     1.0|
|   A|  YY|2003|   5| 7.0|       3|     1.0|
+----+----+----+----+----+--------+--------+



2022-06-30 12:26:33,026 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-30 12:26:33,026 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-30 12:26:33,129 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### 1.3.4 Exp : window has partition, order by and data point

The vtl query:

```text
res := min ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [100]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"min_{target_col_name1}"
new_col_name2=f"min_{target_col_name2}"

df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,min(target_col_name2).over(win_name))
                     
df_min.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|min_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       3|     1.0|
|   A|  XX|2002|   7| 5.0|       3|     1.0|
|   A|  XX|2003|   6| 8.0|       4|     3.0|
|   A|  YY|2000|   9| 3.0|       5|     2.0|
|   A|  YY|2001|   5| 4.0|       5|     2.0|
|   A|  YY|2002|  10| 2.0|       5|     2.0|
|   A|  YY|2003|   5| 7.0|       5|     2.0|
+----+----+----+----+----+--------+--------+



### 1.3.5 Exp : window has partition, order by and range 

The vtl query:

```text
res := min ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [6]:
partition_col_names=["Id_1"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"min_{target_col_name1}"
new_col_name2=f"min_{target_col_name2}"

df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,min(target_col_name2).over(win_name))
                     
df_min.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|min_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  YY|2000|   9| 3.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       3|     1.0|
|   A|  YY|2001|   5| 4.0|       3|     1.0|
|   A|  XX|2002|   7| 5.0|       3|     1.0|
|   A|  YY|2002|  10| 2.0|       3|     1.0|
|   A|  XX|2003|   6| 8.0|       4|     2.0|
|   A|  YY|2003|   5| 7.0|       4|     2.0|
+----+----+----+----+----+--------+--------+



### 1.3.6 Exp6: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc min_Me_1:= min ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [17]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"


new_col_name1=f"min_{target_col_name1}"


df_min=df.withColumn(new_col_name1,min(target_col_name1).over(win_name))                     
df_min.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|min_Me_1|
+----+----+----+----+----+--------+
|   A|  XX|2000|   3| 1.0|       3|
|   A|  XX|2001|   4| 9.0|       3|
|   A|  XX|2002|   7| 5.0|       3|
|   A|  XX|2003|   6| 8.0|       3|
|   A|  YY|2000|   9| 3.0|       9|
|   A|  YY|2001|   5| 4.0|       5|
|   A|  YY|2002|  10| 2.0|       5|
|   A|  YY|2003|   5| 7.0|       5|
+----+----+----+----+----+--------+



## 1.4 max

### 1.4.1 Exp1 : window only has partition clause

The vtl query:

```text
res := max ( ds1 over ( partition by Id_1, Id_2 ) )
```

In [61]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"max_{target_col_name1}"
new_col_name2=f"max_{target_col_name2}"

df_max=df.withColumn(new_col_name1,max(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,max(target_col_name2).over(win_name))
                     
df_max.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|max_Me_1|max_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       7|     9.0|
|   A|  XX|2001|   4| 9.0|       7|     9.0|
|   A|  XX|2002|   7| 5.0|       7|     9.0|
|   A|  XX|2003|   6| 8.0|       7|     9.0|
|   A|  YY|2000|   9| 3.0|      10|     7.0|
|   A|  YY|2001|   5| 4.0|      10|     7.0|
|   A|  YY|2002|  10| 2.0|      10|     7.0|
|   A|  YY|2003|   5| 7.0|      10|     7.0|
+----+----+----+----+----+--------+--------+



### 1.4.2 Exp 2: window has partition and order by

The vtl query:

```text
res := max ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [62]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy("Year")

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"max_{target_col_name1}"
new_col_name2=f"max_{target_col_name2}"

df_max=df.withColumn(new_col_name1,max(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,max(target_col_name2).over(win_name))
                     
df_max.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|max_Me_1|max_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       4|     9.0|
|   A|  XX|2002|   7| 5.0|       7|     9.0|
|   A|  XX|2003|   6| 8.0|       7|     9.0|
|   A|  YY|2000|   9| 3.0|       9|     3.0|
|   A|  YY|2001|   5| 4.0|       9|     4.0|
|   A|  YY|2002|  10| 2.0|      10|     4.0|
|   A|  YY|2003|   5| 7.0|      10|     7.0|
+----+----+----+----+----+--------+--------+



### 1.4.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := max ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [102]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"max_{target_col_name1}"
new_col_name2=f"max_{target_col_name2}"

df_max=df.withColumn(new_col_name1,max(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,max(target_col_name2).over(win_name))
                     
df_max.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|max_Me_1|max_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       7|     9.0|
|   A|  XX|2001|   4| 9.0|       7|     9.0|
|   A|  XX|2002|   7| 5.0|       9|     9.0|
|   A|  XX|2003|   6| 8.0|       9|     9.0|
|   A|  YY|2000|   9| 3.0|      10|     8.0|
|   A|  YY|2001|   5| 4.0|      10|     8.0|
|   A|  YY|2002|  10| 2.0|      10|     7.0|
|   A|  YY|2003|   5| 7.0|      10|     7.0|
+----+----+----+----+----+--------+--------+



### 1.4.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := max ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [103]:
partition_col_names=["Id_1"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"max_{target_col_name1}"
new_col_name2=f"max_{target_col_name2}"

df_max=df.withColumn(new_col_name1,max(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,max(target_col_name2).over(win_name))
                     
df_max.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|max_Me_1|max_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       9|     9.0|
|   A|  YY|2000|   9| 3.0|       9|     9.0|
|   A|  XX|2001|   4| 9.0|      10|     9.0|
|   A|  YY|2001|   5| 4.0|      10|     9.0|
|   A|  XX|2002|   7| 5.0|      10|     9.0|
|   A|  YY|2002|  10| 2.0|      10|     9.0|
|   A|  XX|2003|   6| 8.0|      10|     8.0|
|   A|  YY|2003|   5| 7.0|      10|     8.0|
+----+----+----+----+----+--------+--------+



### 1.4.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc max_Me_1:= max ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [18]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"


new_col_name1=f"max_{target_col_name1}"


df_max=df.withColumn(new_col_name1,max(target_col_name1).over(win_name))                     
df_max.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Year|Me_1|Me_2|max_Me_1|
+----+----+----+----+----+--------+
|   A|  XX|2000|   3| 1.0|       3|
|   A|  XX|2001|   4| 9.0|       4|
|   A|  XX|2002|   7| 5.0|       7|
|   A|  XX|2003|   6| 8.0|       7|
|   A|  YY|2000|   9| 3.0|       9|
|   A|  YY|2001|   5| 4.0|       9|
|   A|  YY|2002|  10| 2.0|      10|
|   A|  YY|2003|   5| 7.0|      10|
+----+----+----+----+----+--------+



## 1.5 avg

### 1.5.1 Exp1 : window only has partition clause

The vtl query:

```text
res := avg ( ds1 over ( partition by Id_1, Id_2 ) )
```


In [104]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"avg_{target_col_name1}"
new_col_name2=f"avg_{target_col_name2}"

df_avg=df.withColumn(new_col_name1,avg(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,avg(target_col_name2).over(win_name))
                     
df_avg.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|avg_Me_1|avg_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|     5.0|    5.75|
|   A|  XX|2001|   4| 9.0|     5.0|    5.75|
|   A|  XX|2002|   7| 5.0|     5.0|    5.75|
|   A|  XX|2003|   6| 8.0|     5.0|    5.75|
|   A|  YY|2000|   9| 3.0|    7.25|     4.0|
|   A|  YY|2001|   5| 4.0|    7.25|     4.0|
|   A|  YY|2002|  10| 2.0|    7.25|     4.0|
|   A|  YY|2003|   5| 7.0|    7.25|     4.0|
+----+----+----+----+----+--------+--------+



### 1.5.2 Exp 2: window has partition and order by

The vtl query:

```text
res := avg ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [105]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy("Year")

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"avg_{target_col_name1}"
new_col_name2=f"avg_{target_col_name2}"

df_avg=df.withColumn(new_col_name1,avg(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,avg(target_col_name2).over(win_name))
                     
df_avg.show()

+----+----+----+----+----+-----------------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|         avg_Me_1|avg_Me_2|
+----+----+----+----+----+-----------------+--------+
|   A|  XX|2000|   3| 1.0|              3.0|     1.0|
|   A|  XX|2001|   4| 9.0|              3.5|     5.0|
|   A|  XX|2002|   7| 5.0|4.666666666666667|     5.0|
|   A|  XX|2003|   6| 8.0|              5.0|    5.75|
|   A|  YY|2000|   9| 3.0|              9.0|     3.0|
|   A|  YY|2001|   5| 4.0|              7.0|     3.5|
|   A|  YY|2002|  10| 2.0|              8.0|     3.0|
|   A|  YY|2003|   5| 7.0|             7.25|     4.0|
+----+----+----+----+----+-----------------+--------+



### 1.5.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := avg ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [106]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"avg_{target_col_name1}"
new_col_name2=f"avg_{target_col_name2}"

df_avg=df.withColumn(new_col_name1,avg(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,avg(target_col_name2).over(win_name))
                     
df_avg.show()

+----+----+----+----+----+-----------------+-----------------+
|Id_1|Id_2|Year|Me_1|Me_2|         avg_Me_1|         avg_Me_2|
+----+----+----+----+----+-----------------+-----------------+
|   A|  XX|2000|   3| 1.0|4.666666666666667|              5.0|
|   A|  XX|2001|   4| 9.0|              5.0|             5.75|
|   A|  XX|2002|   7| 5.0|              5.8|              5.2|
|   A|  XX|2003|   6| 8.0|              6.2|              5.8|
|   A|  YY|2000|   9| 3.0|              7.4|              4.4|
|   A|  YY|2001|   5| 4.0|              7.0|              4.8|
|   A|  YY|2002|  10| 2.0|             7.25|              4.0|
|   A|  YY|2003|   5| 7.0|6.666666666666667|4.333333333333333|
+----+----+----+----+----+-----------------+-----------------+



### 1.5.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := avg ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [107]:
partition_col_names=["Id_1"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"avg_{target_col_name1}"
new_col_name2=f"avg_{target_col_name2}"

df_avg=df.withColumn(new_col_name1,avg(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,avg(target_col_name2).over(win_name))
                     
df_avg.show()

+----+----+----+----+----+-----------------+-----------------+
|Id_1|Id_2|Year|Me_1|Me_2|         avg_Me_1|         avg_Me_2|
+----+----+----+----+----+-----------------+-----------------+
|   A|  XX|2000|   3| 1.0|             5.25|             4.25|
|   A|  YY|2000|   9| 3.0|             5.25|             4.25|
|   A|  XX|2001|   4| 9.0|6.333333333333333|              4.0|
|   A|  YY|2001|   5| 4.0|6.333333333333333|              4.0|
|   A|  XX|2002|   7| 5.0|6.166666666666667|5.833333333333333|
|   A|  YY|2002|  10| 2.0|6.166666666666667|5.833333333333333|
|   A|  XX|2003|   6| 8.0|              7.0|              5.5|
|   A|  YY|2003|   5| 7.0|              7.0|              5.5|
+----+----+----+----+----+-----------------+-----------------+



### 1.5.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc max_Me_1:= avg ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [19]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"


new_col_name1=f"avg_{target_col_name1}"


df_max=df.withColumn(new_col_name1,avg(target_col_name1).over(win_name))                     
df_max.show()

+----+----+----+----+----+-----------------+
|Id_1|Id_2|Year|Me_1|Me_2|         avg_Me_1|
+----+----+----+----+----+-----------------+
|   A|  XX|2000|   3| 1.0|              3.0|
|   A|  XX|2001|   4| 9.0|              3.5|
|   A|  XX|2002|   7| 5.0|4.666666666666667|
|   A|  XX|2003|   6| 8.0|              5.0|
|   A|  YY|2000|   9| 3.0|              9.0|
|   A|  YY|2001|   5| 4.0|              7.0|
|   A|  YY|2002|  10| 2.0|              8.0|
|   A|  YY|2003|   5| 7.0|             7.25|
+----+----+----+----+----+-----------------+



## 1.6 median


### 1.6.1 Exp1 : window only has partition clause

The vtl query:

```text
res := median ( ds1 over ( partition by Id_1, Id_2 ) )
```


percentile_approx with partitionBy followed by orderBy will do rolling median. To have median of each partition, you must only have partitionBy in your window definition.

In [5]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
MEDIAN_PERCENTILE=0.5
PRECISION=10000000
win_name=Window.partitionBy(partition_col_names)


target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"median_{target_col_name1}"
new_col_name2=f"median_{target_col_name2}"

df_median=df.withColumn(new_col_name1,percentile_approx(target_col_name1,MEDIAN_PERCENTILE,PRECISION).over(win_name))\
        .withColumn(new_col_name2,percentile_approx(target_col_name2,MEDIAN_PERCENTILE,PRECISION).over(win_name))
                     

df_median.show()

+----+----+----+----+----+-----------+-----------+
|Id_1|Id_2|Year|Me_1|Me_2|median_Me_1|median_Me_2|
+----+----+----+----+----+-----------+-----------+
|   A|  XX|2000|   3| 1.0|          4|        5.0|
|   A|  XX|2001|   4| 9.0|          4|        5.0|
|   A|  XX|2002|   7| 5.0|          4|        5.0|
|   A|  XX|2003|   6| 8.0|          4|        5.0|
|   A|  YY|2000|   9| 3.0|          5|        3.0|
|   A|  YY|2001|   5| 4.0|          5|        3.0|
|   A|  YY|2002|  10| 2.0|          5|        3.0|
|   A|  YY|2003|   5| 7.0|          5|        3.0|
+----+----+----+----+----+-----------+-----------+



### 1.6.2 Exp 2: window has partition and order by

The vtl query:

```text
res := median ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [6]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
MEDIAN_PERCENTILE=0.5
PRECISION=10000000
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)


target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"median_{target_col_name1}"
new_col_name2=f"median_{target_col_name2}"

df_median=df.withColumn(new_col_name1,percentile_approx(target_col_name1,MEDIAN_PERCENTILE,PRECISION).over(win_name))\
        .withColumn(new_col_name2,percentile_approx(target_col_name2,MEDIAN_PERCENTILE,PRECISION).over(win_name))
                     

df_median.show()

+----+----+----+----+----+-----------+-----------+
|Id_1|Id_2|Year|Me_1|Me_2|median_Me_1|median_Me_2|
+----+----+----+----+----+-----------+-----------+
|   A|  XX|2000|   3| 1.0|          3|        1.0|
|   A|  XX|2001|   4| 9.0|          3|        1.0|
|   A|  XX|2002|   7| 5.0|          4|        5.0|
|   A|  XX|2003|   6| 8.0|          4|        5.0|
|   A|  YY|2000|   9| 3.0|          9|        3.0|
|   A|  YY|2001|   5| 4.0|          5|        3.0|
|   A|  YY|2002|  10| 2.0|          9|        3.0|
|   A|  YY|2003|   5| 7.0|          5|        3.0|
+----+----+----+----+----+-----------+-----------+



### 1.6.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := median ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [9]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]
MEDIAN_PERCENTILE=0.5
PRECISION=10000000
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)


target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"median_{target_col_name1}"
new_col_name2=f"median_{target_col_name2}"

df_median=df.withColumn(new_col_name1,percentile_approx(target_col_name1,MEDIAN_PERCENTILE,PRECISION).over(win_name))\
        .withColumn(new_col_name2,percentile_approx(target_col_name2,MEDIAN_PERCENTILE,PRECISION).over(win_name))
                     

df_median.show()

+----+----+----+----+----+-----------+-----------+
|Id_1|Id_2|Year|Me_1|Me_2|median_Me_1|median_Me_2|
+----+----+----+----+----+-----------+-----------+
|   A|  XX|2000|   3| 1.0|          4|        5.0|
|   A|  XX|2001|   4| 9.0|          4|        5.0|
|   A|  XX|2002|   7| 5.0|          6|        5.0|
|   A|  XX|2003|   6| 8.0|          6|        5.0|
|   A|  YY|2000|   9| 3.0|          7|        4.0|
|   A|  YY|2001|   5| 4.0|          6|        4.0|
|   A|  YY|2002|  10| 2.0|          5|        3.0|
|   A|  YY|2003|   5| 7.0|          5|        4.0|
+----+----+----+----+----+-----------+-----------+



### 1.6.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := median ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [10]:
partition_col_names=["Id_1"]
order_col_names=["Year"]
MEDIAN_PERCENTILE=0.5
PRECISION=10000000
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)


target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"median_{target_col_name1}"
new_col_name2=f"median_{target_col_name2}"

df_median=df.withColumn(new_col_name1,percentile_approx(target_col_name1,MEDIAN_PERCENTILE,PRECISION).over(win_name))\
        .withColumn(new_col_name2,percentile_approx(target_col_name2,MEDIAN_PERCENTILE,PRECISION).over(win_name))
                     

df_median.show()

+----+----+----+----+----+-----------+-----------+
|Id_1|Id_2|Year|Me_1|Me_2|median_Me_1|median_Me_2|
+----+----+----+----+----+-----------+-----------+
|   A|  XX|2000|   3| 1.0|          4|        3.0|
|   A|  YY|2000|   9| 3.0|          4|        3.0|
|   A|  XX|2001|   4| 9.0|          5|        3.0|
|   A|  YY|2001|   5| 4.0|          5|        3.0|
|   A|  XX|2002|   7| 5.0|          5|        5.0|
|   A|  YY|2002|  10| 2.0|          5|        5.0|
|   A|  XX|2003|   6| 8.0|          6|        5.0|
|   A|  YY|2003|   5| 7.0|          6|        5.0|
+----+----+----+----+----+-----------+-----------+



### 1.6.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc median_Me_1:= median ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [20]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
MEDIAN_PERCENTILE=0.5
PRECISION=10000000
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)


target_col_name1="Me_1"


new_col_name1=f"median_{target_col_name1}"


df_median=df.withColumn(new_col_name1,percentile_approx(target_col_name1,MEDIAN_PERCENTILE,PRECISION).over(win_name))
                     

df_median.show()

+----+----+----+----+----+-----------+
|Id_1|Id_2|Year|Me_1|Me_2|median_Me_1|
+----+----+----+----+----+-----------+
|   A|  XX|2000|   3| 1.0|          3|
|   A|  XX|2001|   4| 9.0|          3|
|   A|  XX|2002|   7| 5.0|          4|
|   A|  XX|2003|   6| 8.0|          4|
|   A|  YY|2000|   9| 3.0|          9|
|   A|  YY|2001|   5| 4.0|          5|
|   A|  YY|2002|  10| 2.0|          9|
|   A|  YY|2003|   5| 7.0|          5|
+----+----+----+----+----+-----------+



## 1.7 stddev_pop

An example of steddev_pop and steddev_sample

https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/variance-standard-deviation-sample/a/population-and-sample-standard-deviation-review

The operator returns the “population standard deviation” of the input values.

### 1.7.1 Exp1 : window only has partition clause

The vtl query:

```text
res := stddev_pop ( ds1 over ( partition by Id_1, Id_2 ) )
```


In [78]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_pop_{target_col_name1}"
new_col_name2=f"std_pop_{target_col_name2}"

df_std_pop=df.withColumn(new_col_name1,stddev_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_pop(target_col_name2).over(win_name))
df_std_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      std_pop_Me_1|      std_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|1.5811388300841895| 3.112474899497183|
|   A|  XX|2001|   4| 9.0|1.5811388300841895| 3.112474899497183|
|   A|  XX|2002|   7| 5.0|1.5811388300841895| 3.112474899497183|
|   A|  XX|2003|   6| 8.0|1.5811388300841895| 3.112474899497183|
|   A|  YY|2000|   9| 3.0| 2.277608394786075|1.8708286933869707|
|   A|  YY|2001|   5| 4.0| 2.277608394786075|1.8708286933869707|
|   A|  YY|2002|  10| 2.0| 2.277608394786075|1.8708286933869707|
|   A|  YY|2003|   5| 7.0| 2.277608394786075|1.8708286933869707|
+----+----+----+----+----+------------------+------------------+



### 1.7.2 Exp 2: window has partition and order by

The vtl query:

```text
res := stddev_pop ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [79]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_pop_{target_col_name1}"
new_col_name2=f"std_pop_{target_col_name2}"

df_std_pop=df.withColumn(new_col_name1,stddev_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_pop(target_col_name2).over(win_name))
df_std_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      std_pop_Me_1|      std_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|               0.0|               0.0|
|   A|  XX|2001|   4| 9.0|               0.5|               4.0|
|   A|  XX|2002|   7| 5.0| 1.699673171197595| 3.265986323710904|
|   A|  XX|2003|   6| 8.0|1.5811388300841895| 3.112474899497183|
|   A|  YY|2000|   9| 3.0|               0.0|               0.0|
|   A|  YY|2001|   5| 4.0|               2.0|               0.5|
|   A|  YY|2002|  10| 2.0| 2.160246899469287| 0.816496580927726|
|   A|  YY|2003|   5| 7.0| 2.277608394786075|1.8708286933869707|
+----+----+----+----+----+------------------+------------------+



### 1.7.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := stddev_pop ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [80]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_pop_{target_col_name1}"
new_col_name2=f"std_pop_{target_col_name2}"

df_std_pop=df.withColumn(new_col_name1,stddev_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_pop(target_col_name2).over(win_name))
df_std_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      std_pop_Me_1|      std_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0| 1.699673171197595| 3.265986323710904|
|   A|  XX|2001|   4| 9.0|1.5811388300841895| 3.112474899497183|
|   A|  XX|2002|   7| 5.0|2.1354156504062622| 2.993325909419153|
|   A|  XX|2003|   6| 8.0|1.7204650534085253|2.3151673805580453|
|   A|  YY|2000|   9| 3.0|1.8547236990991407|   2.0591260281974|
|   A|  YY|2001|   5| 4.0|2.0976176963403033|2.3151673805580453|
|   A|  YY|2002|  10| 2.0| 2.277608394786075|1.8708286933869707|
|   A|  YY|2003|   5| 7.0| 2.357022603955158|2.0548046676563256|
+----+----+----+----+----+------------------+------------------+



### 1.7.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := stddev_pop ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [81]:
partition_col_names=["Id_1"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_pop_{target_col_name1}"
new_col_name2=f"std_pop_{target_col_name2}"

df_std_pop=df.withColumn(new_col_name1,stddev_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_pop(target_col_name2).over(win_name))
df_std_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      std_pop_Me_1|      std_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0| 2.277608394786075| 2.947456530637899|
|   A|  YY|2000|   9| 3.0| 2.277608394786075| 2.947456530637899|
|   A|  XX|2001|   4| 9.0| 2.560381915956203|2.5819888974716116|
|   A|  YY|2001|   5| 4.0| 2.560381915956203|2.5819888974716116|
|   A|  XX|2002|   7| 5.0|1.9507833184532708|2.4094720491334933|
|   A|  YY|2002|  10| 2.0|1.9507833184532708|2.4094720491334933|
|   A|  XX|2003|   6| 8.0|1.8708286933869707|  2.29128784747792|
|   A|  YY|2003|   5| 7.0|1.8708286933869707|  2.29128784747792|
+----+----+----+----+----+------------------+------------------+



### 1.7.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc stddev_pop_Me_1:= stddev_pop ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [22]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"


new_col_name1=f"stddev_pop_{target_col_name1}"


df_std_pop=df.withColumn(new_col_name1,stddev_pop(target_col_name1).over(win_name))                     
df_std_pop.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|   stddev_pop_Me_1|
+----+----+----+----+----+------------------+
|   A|  XX|2000|   3| 1.0|               0.0|
|   A|  XX|2001|   4| 9.0|               0.5|
|   A|  XX|2002|   7| 5.0| 1.699673171197595|
|   A|  XX|2003|   6| 8.0|1.5811388300841895|
|   A|  YY|2000|   9| 3.0|               0.0|
|   A|  YY|2001|   5| 4.0|               2.0|
|   A|  YY|2002|  10| 2.0| 2.160246899469287|
|   A|  YY|2003|   5| 7.0| 2.277608394786075|
+----+----+----+----+----+------------------+



## 1.8 stddev_samp

The operator returns the “sample standard deviation” of the input values.

### 1.8.1 Exp1 : window only has partition clause

The vtl query:

```text
res := stddev_samp ( ds1 over ( partition by Id_1, Id_2 ) )
```

In [82]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Id_3"]
win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_samp_{target_col_name1}"
new_col_name2=f"std_samp_{target_col_name2}"

df_std_samp=df.withColumn(new_col_name1,stddev_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_samp(target_col_name2).over(win_name))
df_std_samp.show()


+----+----+----+----+----+------------------+-----------------+
|Id_1|Id_2|Year|Me_1|Me_2|     std_samp_Me_1|    std_samp_Me_2|
+----+----+----+----+----+------------------+-----------------+
|   A|  XX|2000|   3| 1.0|1.8257418583505536|3.593976442141304|
|   A|  XX|2001|   4| 9.0|1.8257418583505536|3.593976442141304|
|   A|  XX|2002|   7| 5.0|1.8257418583505536|3.593976442141304|
|   A|  XX|2003|   6| 8.0|1.8257418583505536|3.593976442141304|
|   A|  YY|2000|   9| 3.0|2.6299556396765835|2.160246899469287|
|   A|  YY|2001|   5| 4.0|2.6299556396765835|2.160246899469287|
|   A|  YY|2002|  10| 2.0|2.6299556396765835|2.160246899469287|
|   A|  YY|2003|   5| 7.0|2.6299556396765835|2.160246899469287|
+----+----+----+----+----+------------------+-----------------+



### 1.8.2 Exp 2: window has partition and order by

The vtl query:

```text
res := stddev_samp ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [83]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_samp_{target_col_name1}"
new_col_name2=f"std_samp_{target_col_name2}"

df_std_samp=df.withColumn(new_col_name1,stddev_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_samp(target_col_name2).over(win_name))
df_std_samp.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     std_samp_Me_1|     std_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|              null|              null|
|   A|  XX|2001|   4| 9.0|0.7071067811865476| 5.656854249492381|
|   A|  XX|2002|   7| 5.0|2.0816659994661326|               4.0|
|   A|  XX|2003|   6| 8.0|1.8257418583505536| 3.593976442141304|
|   A|  YY|2000|   9| 3.0|              null|              null|
|   A|  YY|2001|   5| 4.0|2.8284271247461903|0.7071067811865476|
|   A|  YY|2002|  10| 2.0|2.6457513110645907|               1.0|
|   A|  YY|2003|   5| 7.0|2.6299556396765835| 2.160246899469287|
+----+----+----+----+----+------------------+------------------+



### 1.8.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := stddev_samp ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [84]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_samp_{target_col_name1}"
new_col_name2=f"std_samp_{target_col_name2}"

df_std_samp=df.withColumn(new_col_name1,stddev_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_samp(target_col_name2).over(win_name))
df_std_samp.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     std_samp_Me_1|     std_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|2.0816659994661326|               4.0|
|   A|  XX|2001|   4| 9.0|1.8257418583505536| 3.593976442141304|
|   A|  XX|2002|   7| 5.0|2.3874672772626644|3.3466401061363023|
|   A|  XX|2003|   6| 8.0|1.9235384061671346| 2.588435821108957|
|   A|  YY|2000|   9| 3.0| 2.073644135332772|2.3021728866442674|
|   A|  YY|2001|   5| 4.0| 2.345207879911715| 2.588435821108957|
|   A|  YY|2002|  10| 2.0|2.6299556396765835| 2.160246899469287|
|   A|  YY|2003|   5| 7.0|2.8867513459481287|2.5166114784235836|
+----+----+----+----+----+------------------+------------------+



### 1.8.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := stddev_samp ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [85]:
partition_col_names=["Id_1"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"std_samp_{target_col_name1}"
new_col_name2=f"std_samp_{target_col_name2}"

df_std_samp=df.withColumn(new_col_name1,stddev_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,stddev_samp(target_col_name2).over(win_name))
df_std_samp.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     std_samp_Me_1|     std_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|2.6299556396765835|3.4034296427770228|
|   A|  YY|2000|   9| 3.0|2.6299556396765835|3.4034296427770228|
|   A|  XX|2001|   4| 9.0|2.8047578623950176|2.8284271247461903|
|   A|  YY|2001|   5| 4.0|2.8047578623950176|2.8284271247461903|
|   A|  XX|2002|   7| 5.0| 2.136976056643281|2.6394443859772205|
|   A|  YY|2002|  10| 2.0| 2.136976056643281|2.6394443859772205|
|   A|  XX|2003|   6| 8.0| 2.160246899469287|2.6457513110645907|
|   A|  YY|2003|   5| 7.0| 2.160246899469287|2.6457513110645907|
+----+----+----+----+----+------------------+------------------+



### 1.8.5 Exp5: window inside a calc statement 


The vtl query:

```text
res := ds1 [ calc stddev_samp_Me_1:= stddev_samp ( Me_1 over ( partition by Id_1,Id_2 order by Year ) )];
```

In [23]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]

win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"


new_col_name1=f"stddev_samp_{target_col_name1}"


df_std_samp=df.withColumn(new_col_name1,stddev_samp(target_col_name1).over(win_name))                     
df_std_samp.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|  stddev_samp_Me_1|
+----+----+----+----+----+------------------+
|   A|  XX|2000|   3| 1.0|              null|
|   A|  XX|2001|   4| 9.0|0.7071067811865476|
|   A|  XX|2002|   7| 5.0|2.0816659994661326|
|   A|  XX|2003|   6| 8.0|1.8257418583505536|
|   A|  YY|2000|   9| 3.0|              null|
|   A|  YY|2001|   5| 4.0|2.8284271247461903|
|   A|  YY|2002|  10| 2.0|2.6457513110645907|
|   A|  YY|2003|   5| 7.0|2.6299556396765835|
+----+----+----+----+----+------------------+



## 1.9 var_pop

The operator returns the “population variance” of the input values

### 1.9.1 Exp1 : window only has partition clause

The vtl query:

```text
res := var_pop ( ds1 over ( partition by Id_1, Id_2 ) )
```



In [4]:
partition_col_names=["Id_1","Id_2"]

win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_pop_{target_col_name1}"
new_col_name2=f"var_pop_{target_col_name2}"

df_var_pop=df.withColumn(new_col_name1,var_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_pop(target_col_name2).over(win_name))
df_var_pop.show()


+----+----+----+----+----+------------------+------------+
|Id_1|Id_2|Year|Me_1|Me_2|      var_pop_Me_1|var_pop_Me_2|
+----+----+----+----+----+------------------+------------+
|   A|  XX|2000|   3| 1.0|2.4999999999999996|      9.6875|
|   A|  XX|2001|   4| 9.0|2.4999999999999996|      9.6875|
|   A|  XX|2002|   7| 5.0|2.4999999999999996|      9.6875|
|   A|  XX|2003|   6| 8.0|2.4999999999999996|      9.6875|
|   A|  YY|2000|   9| 3.0|            5.1875|         3.5|
|   A|  YY|2001|   5| 4.0|            5.1875|         3.5|
|   A|  YY|2002|  10| 2.0|            5.1875|         3.5|
|   A|  YY|2003|   5| 7.0|            5.1875|         3.5|
+----+----+----+----+----+------------------+------------+



### 1.9.2 Exp 2: window has partition and order by

The vtl query:

```text
res := var_pop ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [5]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_pop_{target_col_name1}"
new_col_name2=f"var_pop_{target_col_name2}"

df_var_pop=df.withColumn(new_col_name1,var_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_pop(target_col_name2).over(win_name))
df_var_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      var_pop_Me_1|      var_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|               0.0|               0.0|
|   A|  XX|2001|   4| 9.0|              0.25|              16.0|
|   A|  XX|2002|   7| 5.0| 2.888888888888889|10.666666666666666|
|   A|  XX|2003|   6| 8.0|2.4999999999999996|            9.6875|
|   A|  YY|2000|   9| 3.0|               0.0|               0.0|
|   A|  YY|2001|   5| 4.0|               4.0|              0.25|
|   A|  YY|2002|  10| 2.0| 4.666666666666667|0.6666666666666666|
|   A|  YY|2003|   5| 7.0|            5.1875|               3.5|
+----+----+----+----+----+------------------+------------------+



### 1.9.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := var_pop ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [6]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_pop_{target_col_name1}"
new_col_name2=f"var_pop_{target_col_name2}"

df_var_pop=df.withColumn(new_col_name1,var_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_pop(target_col_name2).over(win_name))
df_var_pop.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|      var_pop_Me_1|      var_pop_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0| 2.888888888888889|10.666666666666666|
|   A|  XX|2001|   4| 9.0|2.4999999999999996|            9.6875|
|   A|  XX|2002|   7| 5.0|              4.56| 8.959999999999999|
|   A|  XX|2003|   6| 8.0|              2.96|              5.36|
|   A|  YY|2000|   9| 3.0|              3.44| 4.239999999999999|
|   A|  YY|2001|   5| 4.0|               4.4|              5.36|
|   A|  YY|2002|  10| 2.0|            5.1875|               3.5|
|   A|  YY|2003|   5| 7.0|5.5555555555555545| 4.222222222222222|
+----+----+----+----+----+------------------+------------------+



### 1.9.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := var_pop ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [7]:
partition_col_names=["Id_1"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_pop_{target_col_name1}"
new_col_name2=f"var_pop_{target_col_name2}"

df_var_pop=df.withColumn(new_col_name1,var_pop(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_pop(target_col_name2).over(win_name))
df_var_pop.show()

+----+----+----+----+----+------------------+-----------------+
|Id_1|Id_2|Year|Me_1|Me_2|      var_pop_Me_1|     var_pop_Me_2|
+----+----+----+----+----+------------------+-----------------+
|   A|  XX|2000|   3| 1.0|            5.1875|           8.6875|
|   A|  YY|2000|   9| 3.0|            5.1875|           8.6875|
|   A|  XX|2001|   4| 9.0| 6.555555555555556|6.666666666666668|
|   A|  YY|2001|   5| 4.0| 6.555555555555556|6.666666666666668|
|   A|  XX|2002|   7| 5.0|3.8055555555555554|5.805555555555556|
|   A|  YY|2002|  10| 2.0|3.8055555555555554|5.805555555555556|
|   A|  XX|2003|   6| 8.0|               3.5|             5.25|
|   A|  YY|2003|   5| 7.0|               3.5|             5.25|
+----+----+----+----+----+------------------+-----------------+



## 1.10 var_samp

The operator returns the “sample variance” of the input values

### 1.10.1 Exp1 : window only has partition clause

The vtl query:

```text
res := var_samp ( ds1 over ( partition by Id_1, Id_2 ) )
```



In [90]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Id_3"]
win_name=Window.partitionBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_samp_{target_col_name1}"
new_col_name2=f"var_samp_{target_col_name2}"

df_var_samp=df.withColumn(new_col_name1,var_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_samp(target_col_name2).over(win_name))
df_var_samp.show()


+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     var_samp_Me_1|     var_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|3.3333333333333326|12.916666666666666|
|   A|  XX|2001|   4| 9.0|3.3333333333333326|12.916666666666666|
|   A|  XX|2002|   7| 5.0|3.3333333333333326|12.916666666666666|
|   A|  XX|2003|   6| 8.0|3.3333333333333326|12.916666666666666|
|   A|  YY|2000|   9| 3.0| 6.916666666666667| 4.666666666666667|
|   A|  YY|2001|   5| 4.0| 6.916666666666667| 4.666666666666667|
|   A|  YY|2002|  10| 2.0| 6.916666666666667| 4.666666666666667|
|   A|  YY|2003|   5| 7.0| 6.916666666666667| 4.666666666666667|
+----+----+----+----+----+------------------+------------------+



### 1.10.2 Exp 2: window has partition and order by

The vtl query:

```text
res := var_pop ( ds1 over ( partition by Id_1, Id_2 order by Year) )
```

In [91]:
partition_col_names=["Id_1","Id_2"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_samp_{target_col_name1}"
new_col_name2=f"var_samp_{target_col_name2}"

df_var_samp=df.withColumn(new_col_name1,var_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_samp(target_col_name2).over(win_name))
df_var_samp.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     var_samp_Me_1|     var_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0|              null|              null|
|   A|  XX|2001|   4| 9.0|               0.5|              32.0|
|   A|  XX|2002|   7| 5.0| 4.333333333333333|              16.0|
|   A|  XX|2003|   6| 8.0|3.3333333333333326|12.916666666666666|
|   A|  YY|2000|   9| 3.0|              null|              null|
|   A|  YY|2001|   5| 4.0|               8.0|               0.5|
|   A|  YY|2002|  10| 2.0|               7.0|               1.0|
|   A|  YY|2003|   5| 7.0| 6.916666666666667| 4.666666666666667|
+----+----+----+----+----+------------------+------------------+



### 1.10.3 Exp : window has partition, order by and data point

The vtl query:

```text
res := var_samp ( ds1 over ( partition by Id_1 order by Id_2 data points between 2 preceding and 2 following) )
```

In [92]:
partition_col_names=["Id_1"]
order_col_names=["Id_2"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rowsBetween(-2,2)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_samp_{target_col_name1}"
new_col_name2=f"var_samp_{target_col_name2}"

df_var_samp=df.withColumn(new_col_name1,var_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_samp(target_col_name2).over(win_name))
df_var_samp.show()

+----+----+----+----+----+------------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|     var_samp_Me_1|     var_samp_Me_2|
+----+----+----+----+----+------------------+------------------+
|   A|  XX|2000|   3| 1.0| 4.333333333333333|              16.0|
|   A|  XX|2001|   4| 9.0|3.3333333333333326|12.916666666666666|
|   A|  XX|2002|   7| 5.0| 5.699999999999999|              11.2|
|   A|  XX|2003|   6| 8.0|               3.7|               6.7|
|   A|  YY|2000|   9| 3.0|               4.3| 5.299999999999999|
|   A|  YY|2001|   5| 4.0|               5.5|               6.7|
|   A|  YY|2002|  10| 2.0| 6.916666666666667| 4.666666666666667|
|   A|  YY|2003|   5| 7.0| 8.333333333333332| 6.333333333333334|
+----+----+----+----+----+------------------+------------------+



### 1.10.4 Exp : window has partition, order by and range 

The vtl query:

```text
res := var_samp ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [93]:
partition_col_names=["Id_1"]
order_col_names=["Year"]
win_name=Window.partitionBy(partition_col_names).orderBy(order_col_names).rangeBetween(-1,1)

target_col_name1="Me_1"
target_col_name2="Me_2"

new_col_name1=f"var_samp_{target_col_name1}"
new_col_name2=f"var_samp_{target_col_name2}"

df_var_samp=df.withColumn(new_col_name1,var_samp(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,var_samp(target_col_name2).over(win_name))
df_var_samp.show()

+----+----+----+----+----+-----------------+------------------+
|Id_1|Id_2|Year|Me_1|Me_2|    var_samp_Me_1|     var_samp_Me_2|
+----+----+----+----+----+-----------------+------------------+
|   A|  XX|2000|   3| 1.0|6.916666666666667|11.583333333333334|
|   A|  YY|2000|   9| 3.0|6.916666666666667|11.583333333333334|
|   A|  XX|2001|   4| 9.0|7.866666666666667| 8.000000000000002|
|   A|  YY|2001|   5| 4.0|7.866666666666667| 8.000000000000002|
|   A|  XX|2002|   7| 5.0|4.566666666666666| 6.966666666666667|
|   A|  YY|2002|  10| 2.0|4.566666666666666| 6.966666666666667|
|   A|  XX|2003|   6| 8.0|4.666666666666667|               7.0|
|   A|  YY|2003|   5| 7.0|4.666666666666667|               7.0|
+----+----+----+----+----+-----------------+------------------+

